In [88]:
#참고링크 https://github.com/allenai/bilm-tf
#참고링크 https://appliedmachinelearning.wordpress.com/2019/11/30/training-elmo-from-scratch-on-custom-data-set-for-generating-embeddings-tensorflow/

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [17]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [18]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [19]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [20]:
X = df_newsgroup['text'].astype(str)
# y = tf.keras.utils.to_categorical(df_newsgroup['target'], num_classes=df_newsgroup['target'].nunique())
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [21]:
x_train=list(X_train)
x_test=list(X_test)

In [22]:
import re
pattern = '[^a-z.0-9 ]'
clr_x_trian=[]
clr_x_test=[]
for sen in x_train:
    clr_x_trian.append(re.sub(pattern, '', sen))
for sen in x_test:
    clr_x_test.append(re.sub(pattern, '', sen))

In [23]:
to_txt=clr_x_trian+clr_x_test

In [24]:
#문장 끝에 온점 추가
for i in range(len(to_txt)):
    to_txt[i]=to_txt[i] + " ."

In [25]:
import numpy as np

train_data=np.array(to_txt)

In [26]:
import os
if not os.path.exists("swb/train"):
    os.makedirs("swb/train")
for i in range(0,train_data.shape[0],6):
    text = "\n".join(train_data[i:i+5].tolist())
    fp = open("swb/train/"+str(i)+".txt","w")
    fp.write(text)
    fp.close()


In [27]:
#소문자
from collections import Counter

texts = " ".join(train_data.tolist())
words = texts.split(" ")
print("Number of tokens in Training data = ",len(words))
dictionary = Counter(words)
print("Size of Vocab",len(dictionary))
sorted_vocab = ["<S>","</S>","<UNK>"]
sorted_vocab.extend([pair[0] for pair in dictionary.most_common()])
 
text = "\n".join(sorted_vocab)
fp = open("swb/vocab.txt","w") #,encoding='UTF-8'
fp.write(text)
fp.close()

Number of tokens in Training data =  2639533
Size of Vocab 150994


from collections import Counter

texts = " ".join(dev_data.tolist())
words = texts.split(" ")
print("Number of tokens in Training data = ",len(words))
dictionary = Counter(words)
print("Size of Vocab",len(dictionary))
sorted_vocab = ["<S>","</S>","<UNK>"]
sorted_vocab.extend([pair[0] for pair in dictionary.most_common()])
 
text = "\n".join(sorted_vocab)
fp = open("swb/vocab.txt","w")
fp.write(text)
fp.close()

option.json 파일 설치

In [1]:
import os
os.getcwd()

'C:\\Users\\user\\Desktop\\bilm-tf-master'

In [2]:
#하이퍼 파라미터 옵션(학습이 끝난 뒤엔 꼭 n_character=262로 해줘야 하므로 재설정. 기본값이 261로 돼있을 것임)
options = {
     'bidirectional': True,

     'char_cnn': {'activation': 'relu',
      'embedding': {'dim': 16},
      'filters': [[1, 32],
       [2, 32],
       [3, 64],
       [4, 128],
       [5, 256],
       [6, 512],
       [7, 1024]],
      'max_characters_per_token': 50,
      'n_characters': 262,
      'n_highway': 2},
    
     'dropout': 0.1,
    
     'lstm': {
      'cell_clip': 3,
      'dim': 4096,
      'n_layers': 2,
      'proj_clip': 3,
      'projection_dim': 512,
      'use_skip_connections': True},
    
     'all_clip_norm_val': 10.0,
    
     'n_epochs': 10,
     'n_train_tokens': 2639533,
     'batch_size': 64,
     'n_tokens_vocab': 150994,
     'unroll_steps': 20,
     'n_negative_samples_batch': 8192,
    }

In [3]:
import json
file_path = "./swb/options.json"
with open(file_path, 'w') as outfile:
    json.dump(options, outfile, indent=4)